# Proposal: Predicting the Effect of Lifestyle Factors on the Income of American Citizens # 
## By Daniel Kouchakkhani, Faye McBride, Katherine Dai, & Lillian Milroy ##

### Adult Data Set Analysis ###

#### Introduction ####

In 1994, the average yearly income of an American citizen was $33,178 (U.S. Bureau of the Census, 1996). There are many factors that determine this income — racism, sexism, and ableism are just a few examples of oppression that unfairly lower the wage of marginalized people. Other factors include a person’s level of education and a person’s marital or relationship status. 

Barry Becker extracted data from a 1994 U.S. census that allows one to compare various factors of a person and whether their yearly income is above or below $50 000, he called this data set the Adult Data Set. This data set was limited to observations that occur more than once in the census with an age older than 16, and an adjusted gross income and hours worked greater than one. 

The Adult Data Set contains the following variables;
* age: continuous.
* workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* finalweight: continuous.
* education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* education-num: continuous.
* marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
* occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
* race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
* sex: Female, Male.
* capital-gain: continuous.
* capital-loss: continuous.
* hours-per-week: continuous.
* native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinidad&Tobago, Peru, Hong, Holland-Netherlands.
* yearly_income: <=50K, >50K.

The Adult Data Set offers numerous predictors to determine the yearly income of an American citizen. We are curious about which one has the most influence. Thus, we will be answering the question “Which variable determines with the most accuracy whether a person’s yearly income is greater than or less than $50 000?”

#### Exploratory Data Analysis ####

We started by reading in the data and exploring relevant summary statistics. We then took steps to make it human-readable, and tidy enough for further analysis.

In [1]:
#install.packages("data.validator")
#install.packages("mltools")
#install.packages("data.table")
#install.packages("caret")
#install.packages("fastDummies")
#install.packages("janitor")
library(tidyverse)
library(ggplot2)
library(dplyr)
library(janitor)
library(data.validator)
library(repr)
library(tidymodels)
library(mltools)
library(data.table)
library(caret)
library(fastDummies)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test


── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::

ERROR: Error in library(fastDummies): there is no package called ‘fastDummies’


In [2]:
# Read in the dataset
income_data <- read_csv("https://raw.githubusercontent.com/Daniel040506/dsci-100-project-template/main/adult.data", col_names = FALSE)
head(income_data, 3)

In [3]:
glimpse(income_data)

In [4]:
# find data dimensions
data_dimensions <- dim(income_data)
data_dimensions

In [5]:
# Rename the columns
colnames(income_data) <- c("age","work_class","final_weight","education","num_years_education","marital_status","occupation","relationship",
                         "race","sex","capital_gain","capital_loss","hours_per_week","native_country","income_above_50K")

In [6]:
# Explore data summary
summary(income_data)

In [7]:
# Replace unknown values with NA
income_data[income_data == "?"] <- "NA"

In [8]:
# How many missing values are there in the dataset?
sum(is.na(income_data))

In [9]:
# change income_above_50K column to binary

income_data["income_above_50K"][income_data["income_above_50K"] == ">50K"] <- "0"
income_data["income_above_50K"][income_data["income_above_50K"] == "<=50K"] <- "1"
income_data$income_above_50K <- as.numeric(as.character(income_data$income_above_50K))

head(income_data, 10)

In [10]:
# find the class of each variable
lapply(income_data, class)

In [11]:
# Get rid of empty data/missing values
income_data_clean <- income_data |>
    remove_empty(whic=c("rows")) |>
    remove_empty(whic=c("cols"))
head(income_data_clean, 3)

We created tables with count and  frequency data for each catrgorical variable, and then used these tables to make graphs. We hoped this would help us determine which categorical variables are suitable for further analysis. 

In [12]:
# create vectors of categorical data

work_class_vector <- income_data$work_class
education_vector <- income_data$education
marital_status_vector <- income_data$marital_status
occupation_vector <- income_data$occupation
relationship_vector <- income_data$relationship
race_vector <- income_data$race
sex_vector <- income_data$sex
native_country_vector <- income_data$native_country
income_above_50K_vector <- income_data$income_above_50K

In [13]:
# Work_class

work_class_count <- as.data.frame(table(work_class_vector))
work_class_count
nrow(work_class_count)

work_class_graph <- ggplot(work_class_count, aes(x = work_class_vector, y = Freq, fill = work_class_vector)) +
    geom_bar(stat = "identity") +
    labs(x = "Work Class (vectorized)", y = "Frequency")
work_class_graph

In [14]:
# Education

education_count <- as.data.frame(table(education_vector))
education_count
nrow(education_count)

education_graph <- ggplot(education_count, aes(x = education_vector, y = Freq, fill = education_vector)) +
    geom_bar(stat = "identity") +
    labs(x = "education (vectorized)", y = "Frequency")
education_graph

In [15]:
# Marital_status

marital_status_count <- as.data.frame(table(marital_status_vector))
marital_status_count
nrow(marital_status_count)

marital_status_graph <- ggplot(marital_status_count, aes(x = marital_status_vector, y = Freq, fill = marital_status_vector)) +
    geom_bar(stat = "identity") +
    labs(x = "Marital Status (vectorized)", y = "Frequency")
marital_status_graph

In [16]:
# occupation
income_data |>
    group_by(occupation) |>
    summarise(n = n()) |>
    mutate(Freq = n/sum(n))

In [17]:
# relationship

relationship_count <- as.data.frame(table(relationship_vector))
relationship_count
nrow(relationship_count)

relationship_graph <- ggplot(relationship_count, aes(x = relationship_vector, y = Freq, fill = relationship_vector)) +
    geom_bar(stat = "identity") +
    labs(x = "Relationship (vectorized)", y = "Frequency")
relationship_graph

In [18]:
# race

race_count <- as.data.frame(table(race_vector))
race_count
nrow(race_count)

race_graph <- ggplot(race_count, aes(x = race_vector, y = Freq, fill = race_vector)) +
    geom_bar(stat = "identity") +
    labs(x = "Race (vectorized)", y = "Frequency")
race_graph

In [19]:
# sex

sex_count <- as.data.frame(table(sex_vector))
sex_count
nrow(sex_count)

sex_graph <- ggplot(sex_count, aes(x = sex_vector, y = Freq, fill = sex_vector)) +
    geom_bar(stat = "identity") +
    labs(x = "Race (vectorized)", y = "Frequency")
sex_graph

In [20]:
# native_country

income_data |>
    group_by(native_country) |>
    summarise(n = n()) |>
    mutate(Freq = n/sum(n))

# We will not be creating a graph for native_country, because there are so many possible values that it would not be a good representation of the data.

#### Methods ####
We decided to use a forward selection model to inform our ultimate feature selection. In order to reduce the likelihood of errors occurring, we also decided to eliminate several variables as contenders at this point based on our analyses thus far before proceeding with forward selection.

We observed that native_country, education, work_class, and occupation are polychotomous variables with many possible values. The scope of our analysis is too broad to include variables that require so much additional processing, so they were eliminated from consideration.

The variables marital_status and relationship tell us the same information. marital_status has one more possible value than relationship, so we decided to keep relationship and eliminate marital_status.

capital_gain and capital_loss are not metrics that align meaningfully with our research goals - we are more interested in exploring how income is related to lifestyle factors as opposed to fluctuation of income based on the sale of property and assets, so wel will not include them in the forward selection model.

We have included the code we plan to use for our forward selection model below - it is still being debugged, but we intend to use the accuracy results to select our predictor variables.

Once we have perfected the forward selection model, we will use the results stored in accuracies to help us choose the best combination of predictors for our response variable, income_above_50K. We will use the tabyl function to further examine the most accurate relationships the model indicates before deciding. Once we know what predictor(s) we are using, we will make a subset of the original income_data dataframe which includes the relevant data, and split it into training and testing sets.

We will then use the training and test data to help us build and train a KNN classifier to predict the behaviour of our response variable. We plan to use a scatterplot to help visualize the classifier, and a cross-validation plot to represent the accuracy of the model.

In [21]:
# forward selection
# subset potential predictors

income_subset <- income_data_clean |>
    select(income_above_50K,
           age,
           num_years_education,
           relationship,
           work_class,
           race,
           sex)
income_subset <- as.data.frame(table(income_subset))
head(income_subset, 3)
possible_predictors <- colnames(income_subset |> select(-income_above_50K))
possible_predictors                               

In [22]:
# Creat dummy variables for categorical variables
dummy_income_data <- dummy_columns(income_subset, select_columns = c('age', 'num_years_education', 'relationship', 'work_class', 'race', 'sex'))

ERROR: Error in dummy_columns(income_subset, select_columns = c("age", "num_years_education", : could not find function "dummy_columns"


In [23]:
# Make an empty tibble called accuracies to store results in
accuracies <- tibble(size = integer(), model_string = character(), accuracy = numeric())
accuracies

size,model_string,accuracy
<int>,<chr>,<dbl>


In [24]:
# Create a formula for the model that will be applied to the predictors

formula <- paste("income_above_50K", "~", paste(possible_predictors, collapse="+"))
formula

[1] "income_above_50K ~ age+num_years_education+relationship+work_class+race+sex+Freq"

In [25]:
# Model specification
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

In [26]:
income_data_clean <- income_data_clean %>%
    mutate(income_above_50K = as.factor(income_above_50K))

head(income_data_clean)


# Create object for 5-fold cross-validation
income_vfold <- vfold_cv(income_data_clean, v = 5, strata = income_above_50K)

age,work_class,final_weight,education,num_years_education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income_above_50K
<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<fct>
39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,1
50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,1
38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,1
53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,1
28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,1
37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,1


In [29]:
# Store the total number of predictors
n_total <- length(possible_predictors)

# Create a vector to store the selected predictors in
selected <- c()

ks <- tibble(neighbors = seq(from = 1, to = 2, by = 1))

In [35]:
# Make an empty tibble called accuracies to store results in
accuracies <- tibble(size = integer(), model_string = character(), accuracy = numeric())

all_models <- list()

# Create a for loop to iterate through all of the predictors
for (i in 1:n_total) {  
    #accs <- list()
    #models <- list()
    for (j in 1:length(possible_predictors)) {  
        new_preds <- c(selected, possible_predictors[[j]])
        model_string <- paste("income_above_50K", "~", paste(new_preds, collapse="+"))
         print(model_string)
        # Recipe for model string   
        income_recipe <- recipe(as.formula(model_string), data = income_data_clean) |>
                            step_scale(all_numeric_predictors()) |>
                            step_center(all_numeric_predictors()) |>
                            step_dummy(all_nominal_predictors())
        
        # tune classifier with predictors and find accuracy for best K
        acc <- workflow() |>
            add_recipe(income_recipe) |>
            add_model(knn_spec) |>
            tune_grid(resamples = income_vfold, grid = ks) |>
            collect_metrics() |>
            filter(.metric == "accuracy") %>%
        mutate(model_string = model_string)
    
        # Use the dataframe made earlier to store the result
        accs[[j]] <- acc
        models[[j]] <- model_string
        all_models[[j]] <- acc
        print(accs)
        
        if (j == 2){break}
    }
    #jstar <- which.max(unlist(accs))
    #accuracies <- accuracies |>
    #    add_row(size = i,
    #            model_string = models[[jstar]],
    #            accuracy = accs[[jstar]])
    #selected <- c(selected, possible_predictors[[jstar]])
    #possible_predictors <- possible_predictors[-jstar]
    break
}
all_models

[1] "income_above_50K ~ num_years_education+relationship"
[[1]]
# A tibble: 2 × 8
  neighbors .metric  .estimator  mean     n std_err .config         model_string
      <dbl> <chr>    <chr>      <dbl> <int>   <dbl> <chr>           <chr>       
1         1 accuracy binary     0.736     5  0.0148 Preprocessor1_… income_abov…
2         2 accuracy binary     0.734     5  0.0118 Preprocessor1_… income_abov…

[1] "income_above_50K ~ num_years_education+work_class"
[[1]]
# A tibble: 2 × 8
  neighbors .metric  .estimator  mean     n std_err .config         model_string
      <dbl> <chr>    <chr>      <dbl> <int>   <dbl> <chr>           <chr>       
1         1 accuracy binary     0.736     5  0.0148 Preprocessor1_… income_abov…
2         2 accuracy binary     0.734     5  0.0118 Preprocessor1_… income_abov…

[[2]]
# A tibble: 2 × 8
  neighbors .metric  .estimator  mean     n std_err .config         model_string
      <dbl> <chr>    <chr>      <dbl> <int>   <dbl> <chr>           <chr>       
1 

neighbors,.metric,.estimator,mean,n,std_err,.config,model_string
<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>,<chr>
1,accuracy,binary,0.7356037,5,0.01478741,Preprocessor1_Model1,income_above_50K ~ num_years_education+relationship
2,accuracy,binary,0.7337306,5,0.01177622,Preprocessor1_Model2,income_above_50K ~ num_years_education+relationship
neighbors,.metric,.estimator,mean,n,std_err,.config,model_string
<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>,<chr>
1,accuracy,binary,0.6347165,5,0.022102617,Preprocessor1_Model1,income_above_50K ~ num_years_education+work_class
2,accuracy,binary,0.6054174,5,0.006131166,Preprocessor1_Model2,income_above_50K ~ num_years_education+work_class


In [36]:
library(readr)

In [38]:
saveRDS(all_models, './model_results.RDS')

In [42]:
sadf <- readRDS("model_results.RDS")
sadf

neighbors,.metric,.estimator,mean,n,std_err,.config,model_string
<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>,<chr>
1,accuracy,binary,0.7356037,5,0.01478741,Preprocessor1_Model1,income_above_50K ~ num_years_education+relationship
2,accuracy,binary,0.7337306,5,0.01177622,Preprocessor1_Model2,income_above_50K ~ num_years_education+relationship
neighbors,.metric,.estimator,mean,n,std_err,.config,model_string
<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>,<chr>
1,accuracy,binary,0.6347165,5,0.022102617,Preprocessor1_Model1,income_above_50K ~ num_years_education+work_class
2,accuracy,binary,0.6054174,5,0.006131166,Preprocessor1_Model2,income_above_50K ~ num_years_education+work_class


In [43]:
bind_rows(sadf)

neighbors,.metric,.estimator,mean,n,std_err,.config,model_string
<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>,<chr>
1,accuracy,binary,0.7356037,5,0.014787411,Preprocessor1_Model1,income_above_50K ~ num_years_education+relationship
2,accuracy,binary,0.7337306,5,0.011776218,Preprocessor1_Model2,income_above_50K ~ num_years_education+relationship
1,accuracy,binary,0.6347165,5,0.022102617,Preprocessor1_Model1,income_above_50K ~ num_years_education+work_class
2,accuracy,binary,0.6054174,5,0.006131166,Preprocessor1_Model2,income_above_50K ~ num_years_education+work_class


In [ ]:
show_notes(.Last.tune.result)

#### Expected Outcomes and Significance ####

We expect that of the variables we are considering as predictors at this point, num_years_education and age will have the most influence on whether someone earns more than or less than $50 000 per year. These findings would stress the importance of ensuring education and various occupation types are accessible to everyone and could inspire change. For instance, universities could lower tuition to make education accessible to people from various financial backgrounds. Further studies could analyze the variables that determine the education or occupation someone can receive. This would pinpoint focal areas in making education and various occupation types more accessible. 


#### References ####

U.S. Bureau of the Census, Current Population Reports, P60-193, Money Income in the United States: 1995 (With Separate Data on Valuation of Noncash Benefits), U.S. Government Printing Office, Washington, DC, 1996.

"Data Science: A First Introduction". T. Timbers, T. Campbell, & M. Lee. Chapter 6.8.2, "Finding a good subset of predictors"; Chapter 6.8.3,  "Forward selection in R". 2022.

Test